# UI-TARS 7B GPU Server for Mac OSWorld

## 🎯 Purpose

Run UI-TARS 7B on **Colab A100 GPU** and connect from your **Mac OSWorld** (already tested and working!).

## Architecture & Data Flow

```
┌─────────────────────────────────────────────────────────────────┐
│                    Complete Workflow                              │
│                                                                   │
│  Colab A100 GPU (This Notebook)        Mac (Your Computer)       │
│  ┌──────────────────────────┐         ┌──────────────────────┐  │
│  │ UI-TARS 7B Model         │         │ OSWorld VM (VMware)  │  │
│  │ Flask Server :9000       │         │ ┌──────────────────┐ │  │
│  │ + ngrok tunnel            │         │ │ Ubuntu Desktop   │ │  │
│  │                           │         │ │ Chrome/Firefox   │ │  │
│  │ 1. Receives screenshot    │◄────────│ │ Captures screen  │ │  │
│  │ 2. Generates action       │         │ └──────────────────┘ │  │
│  │ 3. Returns prediction    │────────►│                      │  │
│  │                           │         │ ┌──────────────────┐ │  │
│  │ ⚠️ NO ACCESS TO RESULTS   │         │ │ Executes action  │ │  │
│  │ (Just inference service)  │         │ │ Saves results     │ │  │
│  └──────────────────────────┘         │ │ - traj.jsonl      │ │  │
│                                        │ │ - result.txt      │ │  │
│                                        │ │ - screenshots/    │ │  │
│                                        │ │ - recording.mp4   │ │  │
│                                        │ └──────────────────┘ │  │
│                                        │                      │  │
│                                        │ 📁 results/gpu_eval/ │  │
│                                        │ (Saved on YOUR Mac)  │  │
│                                        └──────────────────────┘  │
└─────────────────────────────────────────────────────────────────┘

Key Points:
✅ Colab GPU = Inference only (no file access)
✅ Mac OSWorld = Task execution + result storage
✅ Full precision model (no quantization) for best quality
✅ Results saved locally: results/gpu_eval/*/traj.jsonl
```

**Advantage**: Uses your TESTED Mac setup, just with GPU model (1200x faster!)

## 1. Install Dependencies

In [1]:
# Install required packages
%pip install -q --upgrade transformers accelerate
%pip install -q qwen-vl-utils pillow flask pyngrok

print("✅ Dependencies installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 142.3 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 67.7 MB/s eta 0:00:00:00:0100:01
✅ Dependencies installed!


## 2. Configure ngrok Authentication

Set up your ngrok authtoken for stable tunnels.

In [ ]:
from pyngrok import conf
import getpass

# Set your ngrok authtoken
# Get it from: https://dashboard.ngrok.com/get-started/your-authtoken
print("📝 Enter your ngrok authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken):")
NGROK_AUTH_TOKEN = getpass.getpass("Authtoken: ")

if not NGROK_AUTH_TOKEN or len(NGROK_AUTH_TOKEN) < 20:
    raise ValueError("Invalid authtoken. Please get it from https://dashboard.ngrok.com")

conf.get_default().auth_token = NGROK_AUTH_TOKEN
print("✅ ngrok authenticated!")

✅ ngrok authenticated!


## 3. Load ARPO UITARS 7B Model

In [ ]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Check GPU
if not torch.cuda.is_available():
    print("❌ No GPU! Go to: Runtime → Change runtime type → A100 GPU")
    raise RuntimeError("GPU required")

print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB\n")

# Model
MODEL = "Fanbin/ARPO_UITARS1.5_7B"
print(f"📥 Loading {MODEL} (full precision)...")

# Load model (use AutoModelForVision2Seq for generation capabilities)
processor = AutoProcessor.from_pretrained(MODEL, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for better performance on A100
)
model.eval()

print(f"\n✅ Model loaded!")
print(f"💾 GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"🔍 Model type: {type(model).__name__}")
print(f"✅ Has generate(): {hasattr(model, 'generate')}")
print("🚀 Ready for serving!")

✅ GPU: NVIDIA A100-SXM4-40GB
💾 Memory: 39.6 GB

📥 Loading Fanbin/ARPO_UITARS1.5_7B (full precision)...


preprocessor_config.json:   0%|          | 0.00/763 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.70G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


✅ Model loaded!
💾 GPU Memory: 14.43 GB
🔍 Model type: Qwen2_5_VLModel
🚀 Ready for serving!


In [4]:
from flask import Flask, request, jsonify
import base64
from io import BytesIO
import time

app = Flask(__name__)

@app.route('/v1/chat/completions', methods=['POST'])
def chat_completions():
    try:
        data = request.json
        messages = data.get('messages', [])
        max_tokens = min(data.get('max_tokens', 256), 512)
        temperature = data.get('temperature', 0.6)
        
        # Process messages
        model_messages = []
        for msg in messages:
            if msg['role'] == 'system':
                continue
            
            content = msg.get('content', [])
            if isinstance(content, str):
                content = [{"type": "text", "text": content}]
            
            processed_content = []
            for item in content:
                if item['type'] == 'image_url':
                    image_url = item['image_url']['url']
                    if image_url.startswith('data:image'):
                        base64_data = image_url.split(',')[1]
                        image_data = base64.b64decode(base64_data)
                        image = Image.open(BytesIO(image_data))
                        processed_content.append({"type": "image", "image": image})
                else:
                    processed_content.append(item)
            
            model_messages.append({"role": msg['role'], "content": processed_content})
        
        # Generate
        inputs = processor.apply_chat_template(
            model_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to(model.device)
        
        start_time = time.time()
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=temperature > 0,
                temperature=temperature if temperature > 0 else 1.0,
            )
        
        response_text = processor.decode(
            outputs[0][inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )
        
        inference_time = time.time() - start_time
        print(f"[{time.strftime('%H:%M:%S')}] Generated in {inference_time:.2f}s")
        
        return jsonify({
            "id": f"chatcmpl-{int(time.time())}",
            "object": "chat.completion",
            "model": "arpo-uitars-7b",
            "choices": [{
                "index": 0,
                "message": {"role": "assistant", "content": response_text},
                "finish_reason": "stop"
            }],
            "usage": {
                "prompt_tokens": inputs["input_ids"].shape[-1],
                "completion_tokens": len(outputs[0]) - inputs["input_ids"].shape[-1]
            },
            "inference_time": inference_time
        })
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500

@app.route('/v1/models', methods=['GET'])
def list_models():
    return jsonify({"object": "list", "data": [{"id": "arpo-uitars-7b"}]})

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "healthy", "model": "arpo-uitars-7b"})

print("✅ Flask server created!")

✅ Flask server created!


In [ ]:
import threading
from pyngrok import ngrok

# Configure port
port = 9001

# Disconnect any existing ngrok tunnels (safe on Colab)
try:
    ngrok.kill()
    time.sleep(1)
    print("✅ Cleaned up old ngrok tunnels")
except:
    pass

# Start Flask server in background thread
print("🚀 Starting Flask server on port 9001...")

def run_flask():
    try:
        app.run(host='0.0.0.0', port=port, debug=False, use_reloader=False, threaded=True)
    except Exception as e:
        print(f"⚠️  Flask error: {e}")

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Give Flask time to start
time.sleep(3)

# Open ngrok tunnel
print("🌐 Creating ngrok tunnel...")
public_url = ngrok.connect(port).public_url

print("\n" + "="*70)
print("✅ SERVER IS READY!")
print("="*70)
print(f"\n📍 Public URL: {public_url}")
print(f"📍 API Endpoint: {public_url}/v1/chat/completions")
print(f"📍 Health Check: {public_url}/health")
print("\n" + "="*70)
print("\n🔧 COPY THIS URL FOR YOUR MAC:")
print(f'\nGPU_SERVER_URL = "{public_url}"')
print("\n" + "="*70)
print("\n⚠️  Keep this cell running! Don't stop or restart.")
print("⚠️  Minimize this tab and leave it open.")
print("\n" + "="*70)

# Keep alive - simple loop
print("\n📊 Server is running. Waiting for requests...")
print("(You'll see inference logs here as requests come in)\n")

try:
    while True:
        time.sleep(10)  # Check every 10 seconds instead of 1
except KeyboardInterrupt:
    print("\n\n🛑 Server stopped by user.")
except Exception as e:
    print(f"\n\n❌ Server crashed: {e}")

: 

: 

## Usage Instructions

### On Colab (This Notebook):
1. ✅ Run cells 1-5
2. ✅ Cell 5 will keep running (server active)
3. ✅ Copy the `GPU_SERVER_URL` from cell 5 output

### On Your Mac (Terminal or Jupyter):

**Option 1: Use the evaluation notebook**
```python
# In ARPO_OSWorld_Evaluation.ipynb Cell 2:
GPU_SERVER_URL = "YOUR_URL_FROM_ABOVE"  # Paste from Colab
```

**Option 2: Run from terminal**
```bash
cd /Users/hanszhu/Desktop/ARPO_replicate

# Update agent
sed -i '' 's|http://localhost:9000|YOUR_URL_FROM_ABOVE|g' OSWorld/mm_agents/uitars_agent.py

# Run evaluation
cd OSWorld
python run_uitars.py \
    --headless \
    --observation_type screenshot \
    --max_steps 15 \
    --model arpo-uitars-7b \
    --temperature 0.6 \
    --test_all_meta_path ../test_data/osworld_examples/test_10tasks.json \
    --result_dir ../results/gpu_eval/
```

### Expected Performance (Actual Test Results):
- **Inference**: ~2-5 seconds per step (model only)
- **Per task**: ~4-5 minutes (includes VM overhead, network latency, execution)
- **5 tasks**: 22m 40s total
- **10 tasks**: ~45-50 minutes estimated

### Monitor:
- Watch this cell's output for inference logs
- Each request shows: "[HH:MM:SS] Generated in X.XXs"